### 0. Load packages

In [1]:
import os
import sys

import gpflow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

import pickle
import scipy as sp
from scipy.sparse import coo_matrix
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *

sys.path.append('../../mogp_decomposition/')
from mwgp import GPD
from data import load_movielens_data_1m

/home/ubuntu/anaconda3/envs/mogp-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/mogp-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/mogp-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/mogp-dev/lib

### 1. Setting random seeds

In [2]:
np.random.seed(123456)
tf.set_random_seed(123456)

### 2. Load data

In [3]:
ml_triple_store = load_movielens_data_1m('../../data/ML-1M/') 

N = ml_triple_store.shape[0]  # number of triples

# Generate new split: 
# write_out = open('./ml-1m_splits.pkl', 'wb')
# pickle.dump(splits, write_out)
# write_out.close()

# Load existent split: 
read_in = open('../../data/ML-1M/splits.pkl', 'rb')
splits = pickle.load(read_in)
read_in.close()

### 3. Training with all splits

In [ ]:
for cv_id in range(5): 
    # Use the current trunk as test set and the rest as training
    te_ids = splits[cv_id]  
    tr_ids = []
    for i in range(len(splits)): 
        if i != cv_id: 
            tr_ids.append(splits[i])
    tr_ids = np.concatenate(tr_ids)    
    
    # As usual in GP, rescaling the target to be zero mean
    target_scaler = StandardScaler()
    
    # I, J are numbers of users and movies, respectively.  
    # Our model can also handle 3 input sources, i.e., kernels, corresponding to 
    #  tensor decomposition. Since ML-1M is a matrix decomposition task, we set K to None.
    I = ml_triple_store[:, 0].max()+1
    J = ml_triple_store[:, 1].max()+1
    K = None
    
    # Prepare the training and test data: 
    #  The user indices and movie indices are found in the first two columns in the 
    #  triple store. The training target is the rescaled third column, i.e. ratings. 
    X_tr = ml_triple_store[tr_ids][:, 0:2]
    Y_tr = target_scaler.fit_transform(ml_triple_store[tr_ids, 3][:, None]).reshape(-1)

    X_te = ml_triple_store[te_ids][:, 0:2]
    Y_te = target_scaler.transform(ml_triple_store[te_ids, 3][:, None]).reshape(-1)    
    
    # We initialize the latent representations with principal components. 
    #  For that we first generate a full matrix from the triple store.
    X_tr_coo = coo_matrix((Y_tr.reshape(-1), (X_tr[:, 0], X_tr[:, 1])), shape=(I, J))
    X_tr_dense = X_tr_coo.todense()
    #  We take the leading principal components. 
    pca_user = PCA(8)
    pca_item = PCA(8)
    user_pcs = pca_user.fit_transform(X_tr_dense)
    item_pcs = pca_item.fit_transform(X_tr_dense.T)

    # The dictionary of hyper parameters: 
    hyper_params = {'I':I, 'J':J, 'K':K,
                    'emb_sizes': [8, 8],     # the size of the latent representations. 
                    'M': 128,                # the number of inducing point pairs.
                    'emb_reg': 1e-4,         # l2 regularization on representation vectors.
                    'batch_size': 2**16,     # mini batch sizeof training
                    'obs_mean': Y_tr.mean(), # mean of target, which is actually 0.
                    'lr': 1e-2}              # learning rate.
    
    # Initialize the model with hyper parameters: 
    gp_md = GPD(**hyper_params)

    # Specify the path to save trained model
    gp_md.save_path = './ml-1m_M=128_cv'+str(cv_id)+'/'
    
    # Build the model. 
    gp_md.build()
    
    # Option 1: using PCA to initialize the latent representations. 
    # Note: so far we do not yet make use of the principal components as initialization 
    #  of the latent representations. 
    #  In order to achieve that using our current implementation, which only supports random 
    #  initialization, we have to apply a small trick: 
    
    # First we call the save() method of the class, which saves two objects:  
    #  The first object consists of the GP hyper parameters. 
    #  The second object consists of the latentrepresentations. 
    gp_md.save()
    
    # Second, we replace the current random representation with the principal components 
    #  in the gp_md model object. 
    param0 = gp_md.get_weights_params()
    param0[0] = user_pcs  
    param0[1] = item_pcs
    # Note this does not change the parameters in the model since get_weights_params() method
    #  makes a copy of the weights. Therefore: 
    
    # Third, we only overwrite the previously saved weights "model_params.pkl" with the
    #  PCA initialization, while leaving the other object, the GP parameters unchanged. 
    with open('./ml-1m_M=128_cv'+str(cv_id)+'/model_params.pkl', 'wb') as f: 
        pickle.dump(param0, f)
    # Finally, we re-load the entire model, with old GP parameters but updated PC as 
    #  initializations of the latent representations.  
    gp_md.load_params()
    
    # Option 2: Alternatively, one could also simply use the random initialization by simply
    #  ignoring everything after gp_md.build()
        
    # Now we can start training: 
    #  The third and fourth parameters are validation X and Y, which we ommit for now. 
    gp_md.train(X_tr, Y_tr, None, None, n_iter=500)
    
    # Save the model after training. 
    gp_md.save()
    
    # Also save the scaler for this specific split configuration, which we need for evaluation. 
    with open('./ml-1m_scaler_cv'+str(cv_id)+'.pkl', 'wb') as f: 
        pickle.dump(target_scaler, f)
    